In [54]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Attention, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [100]:
# Hyperparameters
num_topics = 10
num_clusters = 2
epochs = 50
batch_size = 16

In [101]:
from sklearn.cluster import KMeans

# Step 1: Taxonomy Building

# Load and preprocess the bug data
Train_bug_data = pd.read_csv("/content/preprocessed_train_data2.csv")  # Replace with your train bug data file
Test_bug_data = pd.read_csv("/content/preprocessed_test_data2.csv")  # Replace with your test bug data file

# Handling missing values in bug descriptions
Train_bug_data = Train_bug_data.dropna(subset=["bug_description"])
Test_bug_data = Test_bug_data.dropna(subset=["bug_description"])

train_bug_descriptions = Train_bug_data["bug_description"]
test_bug_descriptions = Test_bug_data["bug_description"]


In [110]:

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
train_X = vectorizer.fit_transform(train_bug_descriptions)
test_X = vectorizer.transform(test_bug_descriptions)

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

# LDA topic modeling
lda_model = LatentDirichletAllocation(
    n_components=2,  # Adjust the number of topics
    random_state=42,
)
topic_distributions = lda_model.fit_transform(train_X)

# Clustering (k-means)
num_clusters = 2  # Adjust the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(topic_distributions)



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [103]:
# Step 2: Attention-based Classification

# NMF for topic modeling on train data
nmf_model = NMF(n_components=num_topics)
topic_vectors_train = nmf_model.fit_transform(train_X)

# Hierarchical Clustering (Agglomerative) for topic vectors
cluster_model_topics = AgglomerativeClustering(n_clusters=num_clusters)
cluster_labels_topics = cluster_model_topics.fit_predict(topic_vectors_train)

# Scale the input data to make it non-negative
scaler_topics = MinMaxScaler()
scaled_topic_vectors_train = scaler_topics.fit_transform(topic_vectors_train)


In [104]:
# Attention-based classification model
input_layer = Input(shape=(scaled_topic_vectors_train.shape[1],))
x = Dense(128, activation="relu")(input_layer)
x = Dropout(0.5)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
attention = Attention()([x, x])
output_layer = Dense(num_clusters, activation="softmax")(attention)
model = Model(inputs=input_layer, outputs=output_layer)


In [105]:
# Compile the model
model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [106]:
# Train the model
model.fit(scaled_topic_vectors_train, cluster_labels_topics, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/50
693/693 [==============================] - 3s 2ms/step - loss: 0.2021 - accuracy: 0.9372 - val_loss: 0.0415 - val_accuracy: 0.9885
Epoch 2/50
693/693 [==============================] - 1s 2ms/step - loss: 0.0455 - accuracy: 0.9876 - val_loss: 0.0391 - val_accuracy: 0.9899
Epoch 3/50
693/693 [==============================] - 2s 2ms/step - loss: 0.0356 - accuracy: 0.9908 - val_loss: 0.0353 - val_accuracy: 0.9899
Epoch 4/50
693/693 [==============================] - 2s 3ms/step - loss: 0.0367 - accuracy: 0.9908 - val_loss: 0.0329 - val_accuracy: 0.9910
Epoch 5/50
693/693 [==============================] - 2s 2ms/step - loss: 0.0326 - accuracy: 0.9923 - val_loss: 0.0279 - val_accuracy: 0.9924
Epoch 6/50
693/693 [==============================] - 2s 2ms/step - loss: 0.0331 - accuracy: 0.9915 - val_loss: 0.0339 - val_accuracy: 0.9913
Epoch 7/50
693/693 [==============================] - 2s 2ms/step - loss: 0.0287 - accuracy: 0.9929 - val_loss: 0.0266 - val_accuracy: 0.9924
Epoch 

## ---------------------------------

In [69]:
from sklearn.cluster import KMeans

# Step 1: Taxonomy Building

# Load and preprocess the bug data
Train_bug_data = pd.read_csv("/content/preprocessed_train_data2.csv")  # Replace with your train bug data file
Test_bug_data = pd.read_csv("/content/preprocessed_test_data2.csv")  # Replace with your test bug data file

# Handling missing values in bug descriptions
Train_bug_data = Train_bug_data.dropna(subset=["bug_description"])
Test_bug_data = Test_bug_data.dropna(subset=["bug_description"])

train_bug_descriptions = Train_bug_data["bug_description"]
test_bug_descriptions = Test_bug_data["bug_description"]


In [70]:

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
train_X = vectorizer.fit_transform(train_bug_descriptions)
test_X = vectorizer.transform(test_bug_descriptions)

# Convert sparse matrix to dense numpy array
train_X_dense = train_X.toarray()
test_X_dense = test_X.toarray()

# KMeans clustering for train data
kmeans_model = KMeans(n_clusters=2, random_state=42)  # Adjust number of clusters as needed
cluster_labels_train = kmeans_model.fit_predict(train_X_dense)

# Evaluate clustering using silhouette score
silhouette_avg = silhouette_score(train_X_dense, cluster_labels_train)
print(f"Silhouette Score: {silhouette_avg}")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette Score: 0.005275638634996257


In [71]:
# Step 2: Attention-based Classification

# NMF for topic modeling on train data
nmf_model = NMF(n_components=2)  # Adjust number of topics as needed
topic_vectors_train = nmf_model.fit_transform(train_X)

# Scale the input data to make it non-negative
scaler = MinMaxScaler()
scaled_topic_vectors_train = scaler.fit_transform(topic_vectors_train)

In [72]:
# Attention-based classification model
input_layer = Input(shape=(scaled_topic_vectors_train.shape[1],))
x = Dense(64, activation="relu")(input_layer)
x = Dropout(0.2)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(8, activation="relu")(x)
attention = Attention()([x, x])
output_layer = Dense(5, activation="softmax")(attention)  # Adjust output classes based on taxonomy
model = Model(inputs=input_layer, outputs=output_layer)

In [73]:
# Compile the model
model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [74]:

# Train the model
model.fit(scaled_topic_vectors_train, cluster_labels_train, epochs=40, batch_size=32, validation_split=0.2)


Epoch 1/40
347/347 [==============================] - 2s 3ms/step - loss: 0.6616 - accuracy: 0.7710 - val_loss: 0.5112 - val_accuracy: 0.8005
Epoch 2/40
347/347 [==============================] - 1s 2ms/step - loss: 0.5260 - accuracy: 0.7857 - val_loss: 0.5053 - val_accuracy: 0.8005
Epoch 3/40
347/347 [==============================] - 1s 2ms/step - loss: 0.4924 - accuracy: 0.7853 - val_loss: 0.4098 - val_accuracy: 0.8142
Epoch 4/40
347/347 [==============================] - 1s 2ms/step - loss: 0.4085 - accuracy: 0.7951 - val_loss: 0.3737 - val_accuracy: 0.8546
Epoch 5/40
347/347 [==============================] - 1s 2ms/step - loss: 0.3900 - accuracy: 0.8086 - val_loss: 0.3760 - val_accuracy: 0.8240
Epoch 6/40
347/347 [==============================] - 1s 2ms/step - loss: 0.3875 - accuracy: 0.7996 - val_loss: 0.3748 - val_accuracy: 0.8449
Epoch 7/40
347/347 [==============================] - 1s 2ms/step - loss: 0.3794 - accuracy: 0.8211 - val_loss: 0.3620 - val_accuracy: 0.8438
Epoch 

In [75]:
# Save the trained model
model.save("attention_based_classification_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [76]:
# Load the saved model
loaded_model = tf.keras.models.load_model("attention_based_classification_model.h5")

In [77]:
# Test the model on test data
test_topic_vectors = nmf_model.transform(test_X)
test_scaled_topic_vectors = scaler.transform(test_topic_vectors)
test_predictions = np.argmax(loaded_model.predict(test_scaled_topic_vectors), axis=-1)


77/77 [==============================] - 0s 1ms/step


In [107]:
# Map Backend to 0 and Frontend to 1
Test_bug_data["class_name"] = Test_bug_data["class_name"].map({"Backend": 0, "Frontend": 1})

# Print the prediction results one by one
for i in range(len(test_predictions)):
    print("Prediction:", test_predictions[i], "Actual:", Test_bug_data["class_name"].iloc[i])


Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 1.0
Prediction: 1 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: nan
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: nan
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: 1.0
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: nan
Prediction: 0 Actual: 0.0
Prediction: 0 Actual: nan
Prediction: 

In [108]:
# Drop rows with NaN values in "class_name"
Test_bug_data.dropna(subset=["class_name"], inplace=True)

# Ensure that both arrays have the same number of samples
num_samples = min(len(Test_bug_data["class_name"]), len(test_predictions))
Test_bug_data = Test_bug_data[:num_samples]
test_predictions = test_predictions[:num_samples]

# Calculate accuracy
accuracy = accuracy_score(Test_bug_data["class_name"], test_predictions)

# Calculate precision
precision = precision_score(Test_bug_data["class_name"], test_predictions)

# Calculate recall
recall = recall_score(Test_bug_data["class_name"], test_predictions)

# Calculate F1-score
f1 = f1_score(Test_bug_data["class_name"], test_predictions)

# Calculate confusion matrix
conf_matrix = confusion_matrix(Test_bug_data["class_name"], test_predictions)

# Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.5746781115879829
Precision: 0.4090909090909091
Recall: 0.00911854103343465
F1 Score: 0.017839444995044598
Confusion Matrix:
[[1330   13]
 [ 978    9]]


In [80]:
# count the number of correct predictions
correct = 0
for i in range(len(test_predictions)):
    if test_predictions[i] == Test_bug_data["class_name"].iloc[i]:
        correct += 1

# Calculate the accuracy
accuracy = correct / len(test_predictions)
print("Accuracy:", accuracy)

print (correct)
print (len(test_predictions))

Accuracy: 0.5549356223175965
1293
2330
